In [1]:
# Note, Issues with .htm extension starts from https://www.resmigazete.gov.tr/eskiler/2000/06/20000627.htm
# From the first issue to https://www.resmigazete.gov.tr/eskiler/2005/01/20050102.htm, inside links direct to footers in page.
# Example: https://www.resmigazete.gov.tr/eskiler/2004/12/20041231.htm#1 etc.
# After this issue, inside links direct to separate pages https://www.resmigazete.gov.tr/eskiler/2005/01/20050102-1.htm etc.



# Until there, the URL style is https://www.resmigazete.gov.tr/arsiv/23894.pdf where the last number indicates Issue number
# The first issue is https://www.resmigazete.gov.tr/arsiv/1.pdf , in Arabic
# The first Turkish issue is https://www.resmigazete.gov.tr/arsiv/1054.pdf 


# Most of the content is in "a" or "p" attribute. Some instances also have "h1" attribute for the first title. (shown in 20 Feb 2006)

## Working note

Before I use the loop, I am trying to understand how to parse efficiently and effectively. The problem is, subclasses are not explicative in html. There are differences throughout the years. Provided links for the contents might be a footer or a redirection. Here are the examples.

Update: To select example dates, adjust the attributes as shown below.

### Xpaths from sample dates

In [2]:
# Scraper class is migrated to resmigazete_module.py 
# Example usage is shown below

from resmigazete_module import ResmiGazeteScraper



In [3]:

# 2000 test
scraper = ResmiGazeteScraper(year=2000)
scraper.months =[f"{month:02d}" for month in range(6, 13)]
scraper.days =[f"{day:02d}" for day in range(26, 27)]
scraper.scrape()
scraper.save_content_to_file()




https://www.resmigazete.gov.tr/eskiler/2000/06/20000626.htm: Status code 404
Content saved to resmigazete_all/XPaths_resmigazete_2000.txt


In [4]:

# 2007 test
scraper = ResmiGazeteScraper(year=2007)
scraper.months =[f"{month:02d}" for month in range(5, 9)]
scraper.days =[f"{day:02d}" for day in range(26, 27)]
scraper.scrape()
scraper.save_content_to_file()

Content saved to resmigazete_all/XPaths_resmigazete_2007.txt
